In [1]:
import pandas as pd
import json, os

import tensorflow as tf
import numpy as np
import csv
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification #, BertModel, BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from collections import OrderedDict
import time
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, \
roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import Embedding, MaxPool1D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Bidirectional, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.initializers import glorot_uniform, RandomUniform, lecun_uniform, Constant
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPool1D
import tensorflow.keras.backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D

from sklearn.metrics import classification_report

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.svm import SVC  
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.tokenize import word_tokenize
import gensim.downloader
import io
from keras_preprocessing.text import tokenizer_from_json
from gensim.models import Word2Vec
from gensim.models import FastText
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.utils import shuffle

D:\anaconda3\envs\tfgpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set the seeder to have as stable random operations as possible

In [2]:
seed = 123
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [3]:
embedding_algorithms = ["w2v", "ft", "w2v_code", "ft_code", "bert", "codebert"]
embedding_algorithm = embedding_algorithms[5]

In [4]:
representations = ["bow", "sequences"]
representation_form = representations[1]

In [5]:
# binary or multi-class
multi = True

In [6]:
train_w2v = False

Read data

In [7]:
if representation_form == "bow":
    #data = pd.read_csv('bow_data.csv') # bow
    data = pd.read_csv('preprocessed_data.csv')
else:
    data = pd.read_csv('sequences_data.csv') # sequences of tokens

In [8]:
data.head()

,Vulnerability,Category,Length
0,"f""str$id""""str$id""""str$id"" ...",sql_injection,9
1,client.listentcp() proxy = proxy(proxy_...,xsrf,8
2,"from django.http import httpresponse, httpresp...",open_redirect,9
3,"def write_preset(conn, queryin, descriptin):\t...",sql_injection,175
4,update_query = self.up...,sql_injection,14


In [9]:
np.max(data["Length"])

392

In [10]:
if representation_form == "sequences":
    word_counts = data["Vulnerability"].apply(lambda x: len(x.split()))
    max_length = word_counts.max()
    print("Maximum number of words:", max_length)


Maximum number of words: 392


In [11]:
label_frequencies = data['Category'].value_counts()
print("Label Frequencies:\n", label_frequencies)
print("Total samples ", len(data))

Label Frequencies:
 sql_injection            1424
xsrf                      976
command_injection         721
path_disclosure           481
open_redirect             442
remote_code_execution     334
xss                       145
Name: Category, dtype: int64
Total samples  4523


In [12]:
if multi == True:
    n_categories = len(label_frequencies) # 7
    n_categories

Word Embedding

Word2Vec - load pre-trained word2vec embeddings - NL knowledge - static embeddings

In [13]:
if representation_form == "sequences":

    # Download the Punkt tokenizer models if not already downloaded
    nltk.download('punkt')

    tokenized_list = [word_tokenize(sentence) for sentence in data["Vulnerability"].tolist()]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iliaskaloup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
if embedding_algorithm == "w2v" and representation_form == "sequences" and train_w2v == False:

    w2v_vectors = gensim.downloader.load('word2vec-google-news-300')
    dim = w2v_vectors.vector_size

#     fileEmb = 'w2v_embeddingsIters.txt'
#     w2v_vectors.save_word2vec_format(fileEmb, binary=False)
    
#     embeddings_index = {}
#     f = open(os.path.join('', fileEmb), encoding="utf-8")
#     for line in f:    
#         values = line.split()
#         word = values[0]
#         coefs = np.asarray(values[1:])
#         embeddings_index[word] = coefs   
#     f.close()
    
    
    tokenizer_obj = Tokenizer()   
    tokenizer_obj.fit_on_texts(tokenized_list)
    
    tokenizer_json = tokenizer_obj.to_json()
    tokenizerFile = 'w2v_tokenizer.json'
    with io.open(tokenizerFile, 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))
        
    with open(tokenizerFile) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)
        
    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    word_index = tokenizer_obj.word_index
    
    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)
    
    
    num_words = len(word_index) + 1 # +1 for the unknown-zeros
    embedding_matrix = np.zeros((num_words, dim))
    for word, i in word_index.items():
        if i > num_words:
            continue
        #embedding_vector = embeddings_index.get(word)
        embedding_vector = w2v_vectors[word] if word in w2v_vectors else None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
  

In [15]:
if embedding_algorithm == "ft" and representation_form == "sequences":

    fastText_vectors = gensim.downloader.load('fasttext-wiki-news-subwords-300')
    dim = fastText_vectors.vector_size   
    
    tokenizer_obj = Tokenizer()   
    tokenizer_obj.fit_on_texts(tokenized_list)
    
    tokenizer_json = tokenizer_obj.to_json()
    tokenizerFile = 'fast_tokenizer.json'
    with io.open(tokenizerFile, 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))
        
    with open(tokenizerFile) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)
        
    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    word_index = tokenizer_obj.word_index
    
    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)    
    
    num_words = len(word_index) + 1 # +1 for the unknown-zeros
    embedding_matrix = np.zeros((num_words, dim))
    for word, i in word_index.items():
        if i > num_words:
            continue
        #embedding_vector = embeddings_index.get(word)
        embedding_vector = fastText_vectors[word] if word in fastText_vectors else None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
  

Pre-trained Word2Vec embeddings

In [16]:
if embedding_algorithm == "w2v_code" and representation_form == "sequences":

    embeddings_index = {}
    f = open(os.path.join('', 'w2v_embeddings.txt'), encoding="utf-8")
    for line in f:    
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:])
        embeddings_index[word] = coefs   
    f.close() 
    
    dim = 100
    
    tokenizer_obj = Tokenizer()   
    tokenizer_obj.fit_on_texts(tokenized_list)
    
    tokenizer_json = tokenizer_obj.to_json()
    tokenizerFile = 'w2v_new_tokenizer.json'
    with io.open(tokenizerFile, 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))
        
    with open(tokenizerFile) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)
        
    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    word_index = tokenizer_obj.word_index
    
    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)    
    
    num_words = len(word_index) + 1 # +1 for the unknown-zeros
    embedding_matrix = np.zeros((num_words, dim))
    for word, i in word_index.items():
        if i > num_words:
            continue
        #embedding_vector = embeddings_index.get(word)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
  

Pre-trained FastText embeddings

In [17]:
if embedding_algorithm == "ft_code" and representation_form == "sequences":

    embeddings_index = {}
    f = open(os.path.join('', 'fast-text_embeddings.txt'), encoding="utf-8")
    for line in f:    
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:])
        embeddings_index[word] = coefs   
    f.close() 
    
    dim = 100
    
    tokenizer_obj = Tokenizer()   
    tokenizer_obj.fit_on_texts(tokenized_list)
    
    tokenizer_json = tokenizer_obj.to_json()
    tokenizerFile = 'fastCode_tokenizer.json'
    with io.open(tokenizerFile, 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))
        
    with open(tokenizerFile) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)
        
    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    word_index = tokenizer_obj.word_index
    
    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)    
    
    num_words = len(word_index) + 1 # +1 for the unknown-zeros
    embedding_matrix = np.zeros((num_words, dim))
    for word, i in word_index.items():
        if i > num_words:
            continue
        #embedding_vector = embeddings_index.get(word)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
  

BERT - load pre-trained bert embeddings - NL knowledge - contextual embeddings

In [18]:
if embedding_algorithm == "bert" and representation_form == "sequences": 
    model_variation = "bert-base-uncased" # "roberta-base"
    tokenizer = AutoTokenizer.from_pretrained(model_variation)
    bert = TFAutoModel.from_pretrained(model_variation)
    
    bert_embeddings = bert.get_input_embeddings()
    embedding_matrix = bert_embeddings.weights[0].numpy()
    num_words = len(embedding_matrix)
    print(num_words)
    dim = len(embedding_matrix[0])
    print(dim)

In [19]:
if embedding_algorithm == "bert" and representation_form == "sequences":
    sentences = data["Vulnerability"].tolist()
    sequences = [tokenizer.encode(sente, truncation=True, add_special_tokens=False, return_tensors="tf").numpy() for sente in sentences] # Tokenize the complete sentences
    
    lines_pad = []
    for seq in sequences:
        lines_pad.append(seq[0])
    
    lines_pad = pad_sequences(lines_pad, padding = 'post', maxlen = 512)

CodeBERT - load pre-trained codebert embeddings - PL knowledge - contextual embeddings

In [20]:
if embedding_algorithm == "codebert" and representation_form == "sequences": 
    model_variation = "microsoft/codebert-base"
    tokenizer = AutoTokenizer.from_pretrained(model_variation, do_lower_case=True)
    codebert = TFAutoModel.from_pretrained(model_variation)
    
    codebert_embeddings = codebert.get_input_embeddings()
    embedding_matrix = codebert_embeddings.weights[0].numpy()
    num_words = len(embedding_matrix)
    print(num_words)
    dim = len(embedding_matrix[0])
    print(dim)

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


50265
768


In [21]:
def padSequences(sequences, max_len):
    lines_pad = []
    for sequence in sequences:
        seq = sequence['input_ids'].numpy()[0]
        if len(seq) < max_len:
            for i in range(len(seq), max_len):
                seq = np.append(seq, 0)
        lines_pad.append(seq)
    return lines_pad


In [22]:
if embedding_algorithm == "codebert" and representation_form == "sequences": 

    sentences = data["Vulnerability"].tolist()
    sequences = [tokenizer(sente, return_tensors="tf", truncation=True, add_special_tokens=False) for sente in sentences]
    
    lines_pad = padSequences(sequences, 512)
    lines_pad = [arr.tolist() for arr in lines_pad]
    lines_pad = np.array(lines_pad)

Visualize Text Vectors

RNN model, LSTM specifically

In [23]:
def buildLstm(max_len, top_words, dim, seed, embedding_matrix, multi, n_outputs):
    model=Sequential()
    #model.add(Embedding(input_dim=top_words+1, output_dim=dim, input_length=None, mask_zero=True))
    model.add(Embedding(input_dim=top_words, output_dim=dim, input_length=None, weights=[embedding_matrix], mask_zero=True, trainable=False))
    #model.add(SimpleRNN(300, dropout=0.3, stateful=False))
    model.add(LSTM(100, dropout=0.2, return_sequences=True, stateful=False))
    model.add(LSTM(50, dropout=0.1, stateful=False))
    #model.add(Bidirectional(LSTM(300, dropout=0.3, stateful=False)))
    #model.add(GRU(300, dropout=0.3, stateful=False))
    model.add(Activation('relu')) #dropout=0.2, recurrent_dropout=0.2, kernel_constraint=max_norm(3), bias_constraint=max_norm(3)
    model.add(BatchNormalization(momentum=0.0))
    if multi == False:
        model.add(Dense(n_outputs, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam')  
    else: 
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    return model

CNN model 1-d

In [24]:
def buildCnn(max_len, top_words, dim, seed, embedding_matrix, multi, n_outputs):
    cnn_model = Sequential()
    cnn_model.add(Embedding(top_words, dim, input_length=None, weights=[embedding_matrix], mask_zero=True, trainable=False))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 1, activation = 'relu'))
    '''cnn_model.add(MaxPooling1D(pool_size = 5))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))
    cnn_model.add(MaxPooling1D(pool_size = 5))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))'''
    cnn_model.add(GlobalMaxPool1D())
    #cnn_model.add(Dense(units = 128, activation = 'relu'))
    if multi == False:
        cnn_model.add(Dense(n_outputs, activation = 'sigmoid'))
        cnn_model.compile(loss = "binary_crossentropy", optimizer = "adam")
    else: 
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    return cnn_model

Cross-Validation

Binary Classification: Recognition of Injection Vulnerabilities (command_injection and sql_injection merged)

In [25]:
# Define a function to determine if the category is an injection or not
def is_injection(category):
    if category in ['sql_injection', 'command_injection']:
        return 1
    else:
        return 0

if multi == False:
    data['Injection'] = data['Category'].apply(is_injection)

In [26]:
# BoW and ML

def runMLCrossVal_binary(X, y, seed, userModel):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
    
#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)
        
        if userModel == "svm":
            myModel = SVC(kernel='rbf', gamma=100)
        elif userModel == "RF":
            myModel = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')
        elif userModel == "DT":
            myModel = tree.DecisionTreeClassifier(max_depth=120)
        elif userModel == "NB":
            myModel = GaussianNB()

        myModel.fit(X_train, Y_train.ravel())
        
        predictions = myModel.predict(X_test)
        #predScores = myModel.predict_proba(X_test)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions)
        recall=recall_score(Y_test, predictions)
        f1=f1_score(Y_test, predictions)
        roc_auc=roc_auc_score(Y_test, predictions)
        print(confusion_matrix(Y_test, predictions, labels=[0, 1]))
        tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
        acc = ((tp+tn)/(tp+tn+fp+fn))
        print("Accuracy:%.2f%%"%(acc*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print("Roc_Auc score:%.2f%%"%(roc_auc*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        score_dict['roc_auc'] = np.append(score_dict['roc_auc'], roc_auc)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))
    print("roc_auc: %.2f%% (%.2f%%)" % (score_dict['roc_auc'].mean()*100, score_dict['roc_auc'].std()*100))

In [27]:
if multi == False and representation_form == "bow":
    runMLCrossVal_binary(data.iloc[:, 0:bow_size], data["Injection"], seed, "RF")

In [28]:
# Sequences of tokens and DL

def runDLCrossVal_binary(X, y, max_len, num_words, dim, seed, embedding_matrix, userModel, multi):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
    
#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)
        
        if userModel == "cnn":
            myModel = buildCnn(max_len, num_words, dim, seed, embedding_matrix, multi, 1) 
        elif userModel == "lstm":
            myModel = buildLstm(max_len, num_words, dim, seed, embedding_matrix, multi, 1)
        print("model summary\m",myModel.summary())
        csv_logger = CSVLogger('log.csv', append=True, separator=',')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])
        
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        
        scores = myModel.evaluate(X_test, Y_test, verbose=0)
        #predictions = myModel.predict_classes(X_test, verbose=0)
        predictions = (myModel.predict(X_test) > 0.5).astype("int32")
        predScores = myModel.predict(X_test)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions)
        recall=recall_score(Y_test, predictions)
        f1=f1_score(Y_test, predictions)
        roc_auc=roc_auc_score(Y_test, predictions)
        print(confusion_matrix(Y_test, predictions, labels=[0, 1]))
        tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
        acc = ((tp+tn)/(tp+tn+fp+fn))
        print("Accuracy:%.2f%%"%(acc*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print("Roc_Auc score:%.2f%%"%(roc_auc*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        score_dict['roc_auc'] = np.append(score_dict['roc_auc'], roc_auc)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))
    print("roc_auc: %.2f%% (%.2f%%)" % (score_dict['roc_auc'].mean()*100, score_dict['roc_auc'].std()*100))

In [29]:
if multi == False and representation_form == "sequences":
    runDLCrossVal_binary(lines_pad, data["Injection"].values, 512, num_words, dim, seed, embedding_matrix, "lstm", multi)

Multi-class Classification: Categorization of all detected vulnerabilities

In [30]:
if multi == True:
    
    # Convert categories to numerical indexes
    category_numerical_indexes, unique_categories = data["Category"].factorize()

    # Create a dictionary mapping each category to its numerical index
    category_to_index = {category: index for index, category in enumerate(unique_categories)}

    # Update the categories in the DataFrame with their numerical indexes
    data["Category_Index"] = data["Category"].map(category_to_index)


Use ML models and BoW code representation

In [31]:
# No sampling
def runMLCrossVal_multi(X, y, seed, userModel):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
        
        X_train_list = X_train["Vulnerability"].tolist()
        X_test_list = X_test["Vulnerability"].tolist()

        vectorizer = CountVectorizer() #min_df=1
        x_train = vectorizer.fit_transform(X_train_list)
        x_test = vectorizer.transform(X_test_list)
        
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         class_counts = pd.Series(Y_train).value_counts()
#         majority_class = class_counts.idxmax()
#         sampling_strategy = {
#             majority_class: class_counts.iloc[1]  # Set the second largest class as the target count
#         }
        
#         #X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         x_resampled, y_resampled = RandomUnderSampler(random_state=seed, sampling_strategy=sampling_strategy).fit_resample(x_train, Y_train)
    
#         #shuffle dataset
#         x_train, Y_train = shuffle(x_resampled, y_resampled, random_state=seed)
#         x_train = pd.DataFrame(x_train)
        
#         # Convert dataframes to numpy arrays if they are not already
#         x_train = np.array(x_train)
#         Y_train = np.array(Y_train)

        
        if userModel == "SVM":
            myModel = SVC(kernel='rbf', gamma=100)
        elif userModel == "RF":
            myModel = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')
        elif userModel == "DT":
            myModel = tree.DecisionTreeClassifier(max_depth=120)
        elif userModel == "NB":
            myModel = GaussianNB()
            
        myModel.fit(x_train, Y_train.ravel())
        
        predictions = myModel.predict(x_test)
        #predScores = myModel.predict_proba(X_test)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions, average='macro')
        recall=recall_score(Y_test, predictions, average='macro')
        f1=f1_score(Y_test, predictions, average='macro')
        conf_matrix = confusion_matrix(Y_test, predictions)
        print(conf_matrix)
        #tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
        #acc = ((tp+tn)/(tp+tn+fp+fn))
        print("Accuracy:%.2f%%"%(accuracy*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [32]:
# # under-sampling
# def runMLCrossVal_multi(X, y, seed, userModel):
#     ############## cross validation
#     scores=['accuracy', 'precision', 'recall', 'f1']
#     values = [np.array([]) for i in range(0, len(scores))]
#     score_dict = OrderedDict(zip(scores, values))
#     k=10
#     f=0
#     kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
#     nb_epoch = 100
#     BS = 64
#     print("Training...")
#     milli_sec1 = int(round(time.time() * 1000))
    
#     for train_index, test_index in kfold.split(X, y):
#         f = f + 1
#         print('fold number= ',f)
#         X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
#         Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
        
#         X_train_list = X_train["Vulnerability"].tolist()
#         X_test_list = X_test["Vulnerability"].tolist()

#         vectorizer = CountVectorizer() #min_df=1
#         x_train = vectorizer.fit_transform(X_train_list)
#         x_test = vectorizer.transform(X_test_list)
        
#         # Apply under-sampling with the specified strategy
#         class_counts = pd.Series(Y_train).value_counts()
#         majority_class = class_counts.idxmax() # majority class
#         target_count = class_counts.iloc[1]  # Number of samples in the second largest class
        
#         sampling_strategy = {majority_class: target_count}
        
#         rus = RandomUnderSampler(random_state=seed, sampling_strategy=sampling_strategy)
#         x_train_resampled, Y_train_resampled = rus.fit_resample(x_train, Y_train)        
#         # Shuffle the resampled data while preserving the correspondence between features and labels
#         x_train_resampled, Y_train_resampled = shuffle(x_train_resampled, Y_train_resampled, random_state=seed)
        
#         x_train = x_train_resampled
#         Y_train = Y_train_resampled
        
#         if userModel == "SVM":
#             myModel = SVC(kernel='rbf', gamma=100)
#         elif userModel == "RF":
#             myModel = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')
#         elif userModel == "DT":
#             myModel = tree.DecisionTreeClassifier(max_depth=120)
#         elif userModel == "NB":
#             myModel = GaussianNB()
            
#         myModel.fit(x_train, Y_train)
        
#         predictions = myModel.predict(x_test)
#         #predScores = myModel.predict_proba(X_test)
#         accuracy=accuracy_score(Y_test, predictions)
#         precision=precision_score(Y_test, predictions, average='macro')
#         recall=recall_score(Y_test, predictions, average='macro')
#         f1=f1_score(Y_test, predictions, average='macro')
#         conf_matrix = confusion_matrix(Y_test, predictions)
#         print(conf_matrix)
#         #tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
#         #acc = ((tp+tn)/(tp+tn+fp+fn))
#         print("Accuracy:%.2f%%"%(accuracy*100))
#         print("Precision:%.2f%%"%(precision*100))
#         print("Recall:%.2f%%"%(recall*100))
#         print("F1 score:%.2f%%"%(f1*100))
#         print(classification_report(Y_test, predictions))
#         del myModel
#         score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
#         score_dict['precision'] = np.append(score_dict['precision'], precision)
#         score_dict['recall'] = np.append(score_dict['recall'], recall)
#         score_dict['f1'] = np.append(score_dict['f1'], f1)
        
#     milli_sec2 = int(round(time.time() * 1000))
#     print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
#     print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
#     print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
#     print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
#     print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [33]:
if multi == True and representation_form == "bow":
    runMLCrossVal_multi(pd.DataFrame(data["Vulnerability"]), data["Category_Index"], seed, "RF")

Use DL models and sequences of tokens code representation

In [34]:
# no sampling
def runDLCrossVal_multi(X, y, max_len, num_words, dim, seed, embedding_matrix, userModel, multi):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]
        
        if userModel == "cnn":
            myModel = buildCnn(max_len, num_words, dim, seed, embedding_matrix, multi, n_categories) 
        elif userModel == "lstm":
            myModel = buildLstm(max_len, num_words, dim, seed, embedding_matrix, multi, n_categories)
        print("model summary\m",myModel.summary())
        csv_logger = CSVLogger('log.csv', append=True, separator=',')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])
        
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        
        scores = myModel.evaluate(X_test, Y_test, verbose=0)
        #predictions = myModel.predict_classes(X_test, verbose=0)
        predScores = myModel.predict(X_test)
        predictions = np.argmax(predScores, axis=1)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions, average='macro')
        recall=recall_score(Y_test, predictions, average='macro')
        f1=f1_score(Y_test, predictions, average='macro')
        conf_matrix = confusion_matrix(Y_test, predictions)
        print(conf_matrix)
        print("Accuracy:%.2f%%"%(accuracy*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [35]:
# over and undersampling
def runDLCrossVal_multi(X, y, max_len, num_words, dim, seed, embedding_matrix, userModel, multi):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]

        # Apply under-sampling with the specified strategy
        class_counts = pd.Series(Y_train).value_counts()
        majority_class = class_counts.idxmax() # majority class
        target_count = class_counts.iloc[1]  # Number of samples in the second largest class
        
        # under
        sampling_strategy = {majority_class: target_count}        
        rus = RandomUnderSampler(random_state=seed, sampling_strategy=sampling_strategy)
        x_train_resampled, Y_train_resampled = rus.fit_resample(X_train, Y_train)  
        
        # over
        for mc in range(2, len(class_counts)):
            minority_class = class_counts.iloc[mc]
            sampling_strategy = {minority_class: target_count}
            ros = RandomOverSampler(random_state=seed, sampling_strategy=sampling_strategy)
            x_train_resampled, Y_train_resampled = rus.fit_resample(x_train_resampled, Y_train_resampled)
        
        # Shuffle the resampled data while preserving the correspondence between features and labels
        x_train_resampled, Y_train_resampled = shuffle(x_train_resampled, Y_train_resampled, random_state=seed)
        
        X_train = x_train_resampled
        Y_train = Y_train_resampled
        
        if userModel == "cnn":
            myModel = buildCnn(max_len, num_words, dim, seed, embedding_matrix, multi, n_categories) 
        elif userModel == "lstm":
            myModel = buildLstm(max_len, num_words, dim, seed, embedding_matrix, multi, n_categories)
        print("model summary\m",myModel.summary())
        csv_logger = CSVLogger('log.csv', append=True, separator=',')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])
        
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        
        scores = myModel.evaluate(X_test, Y_test, verbose=0)
        #predictions = myModel.predict_classes(X_test, verbose=0)
        predScores = myModel.predict(X_test)
        predictions = np.argmax(predScores, axis=1)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions, average='macro')
        recall=recall_score(Y_test, predictions, average='macro')
        f1=f1_score(Y_test, predictions, average='macro')
        conf_matrix = confusion_matrix(Y_test, predictions)
        print(conf_matrix)
        print("Accuracy:%.2f%%"%(accuracy*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [36]:
if multi == True and representation_form == "sequences" and train_w2v == False:
    runDLCrossVal_multi(lines_pad, data["Category_Index"].values, 512, num_words, dim, seed, embedding_matrix, "lstm", multi)

Training...
fold number=  1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 768)         38603520  
_________________________________________________________________
lstm (LSTM)                  (None, None, 100)         347600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
activation (Activation)      (None, 50)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 50)                200       
_________________________________________________________________
dense (Dense)                (None, 7)                 357       
Total params: 38,981,877
Trainable params: 378,257
Non-trainable params: 38,603,620
__________

58/58 [==============================] - 4s 68ms/step - loss: 0.2927 - val_loss: 0.6377

Epoch 00010: val_loss did not improve from 0.61562
Epoch 11/100
58/58 [==============================] - 4s 69ms/step - loss: 0.2652 - val_loss: 0.6517

Epoch 00011: val_loss did not improve from 0.61562
Epoch 12/100
58/58 [==============================] - 4s 70ms/step - loss: 0.2300 - val_loss: 0.6171 1s - loss:  - ETA: 0s - 

Epoch 00012: val_loss did not improve from 0.61562
Epoch 13/100
58/58 [==============================] - 4s 67ms/step - loss: 0.1899 - val_loss: 0.6345

Epoch 00013: val_loss did not improve from 0.61562
Epoch 14/100
58/58 [==============================] - 4s 67ms/step - loss: 0.1598 - val_loss: 0.6191.159 - ETA: 0s - loss: 0.1

Epoch 00014: val_loss did not improve from 0.61562
Epoch 15/100
58/58 [==============================] - 4s 67ms/step - loss: 0.1474 - val_loss: 0.6694s: 

Epoch 00015: val_loss did not improve from 0.61562
Epoch 16/100
58/58 [=====================

ResourceExhaustedError: OOM when allocating tensor with shape[50265,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:Add]

One more option for training your own word2vec vectors

In [ ]:
def trainWord2Vec(tokenized_list, max_length, dim):
    # this should be executed only in the training set during cross-validation
    w2v_model = Word2Vec(sentences=tokenized_list, vector_size=dim, window=5, min_count=1, workers=4, epochs=4)
    #w2v_model.save("python_word2vec.model")
    #w2v_model = Word2Vec.load("python_word2vec.model")

    #     fileEmb = 'w2v_embeddingsIters.txt'
    #     w2v_vectors.save_word2vec_format(fileEmb, binary=False)

    #     embeddings_index = {}
    #     f = open(os.path.join('', fileEmb), encoding="utf-8")
    #     for line in f:    
    #         values = line.split()
    #         word = values[0]
    #         coefs = np.asarray(values[1:])
    #         embeddings_index[word] = coefs   
    #     f.close()


    tokenizer_obj = Tokenizer()   
    tokenizer_obj.fit_on_texts(tokenized_list)

    tokenizer_json = tokenizer_obj.to_json()
    tokenizerFile = 'w2v_new_tokenizer.json'
    with io.open(tokenizerFile, 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))

    with open(tokenizerFile) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)

    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    word_index = tokenizer_obj.word_index

    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)


    num_words = len(word_index) + 1 # +1 for the unknown-zeros
    embedding_matrix = np.zeros((num_words, dim))
    for word, i in word_index.items():
        if i > num_words:
            continue
        #embedding_vector = embeddings_index.get(word)
        embedding_vector = w2v_model.wv[word] if word in w2v_model.wv else None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return lines_pad, embedding_matrix, num_words, tokenizerFile


In [ ]:
def testWord2Vec(tokenized_list, tokenizer_path, max_length):
    
    with open(tokenizer_path) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)
        
    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    
    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)
    
    return lines_pad

In [ ]:
def runDLCrossValW2v_multi(X, y, max_len, dim, seed, userModel):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]
        
        X_train, embedding_matrix, num_words, tokenizerFile = trainWord2Vec(X_train, max_len, dim)
        
        X_test = testWord2Vec(X_test, tokenizerFile, max_len)
        
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
    
#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)
        
        if userModel == "cnn":
            myModel = buildCnn(max_len, num_words, dim, seed, embedding_matrix, multi, n_categories) 
        elif userModel == "lstm":
            myModel = buildLstm(max_len, num_words, dim, seed, embedding_matrix, multi, n_categories)
        print("model summary\m",myModel.summary())
        csv_logger = CSVLogger('log.csv', append=True, separator=',')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        
        history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])
        
        
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        
        scores = myModel.evaluate(X_test, Y_test, verbose=0)
        #predictions = myModel.predict_classes(X_test, verbose=0)
        predScores = myModel.predict(X_test)
        predictions = np.argmax(predScores, axis=1)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions, average='macro')
        recall=recall_score(Y_test, predictions, average='macro')
        f1=f1_score(Y_test, predictions, average='macro')
        conf_matrix = confusion_matrix(Y_test, predictions)
        print(conf_matrix)
        print("Accuracy:%.2f%%"%(accuracy*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [ ]:
if train_w2v == True and multi == True and representation_form == "sequences":
    dim = 300
    runDLCrossValW2v_multi(np.array(tokenized_list), data["Category_Index"].values, 512, dim, seed, "lstm")